In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings; warnings.simplefilter('ignore', FutureWarning)
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import pymongo
from pymongo import MongoClient
from avian_vocalizations.evaluator import ParamSpace, EvaluatorFactory
from hyperopt.mongoexp import MongoTrials, MongoCtrl, MongoJobs, MongoWorker
from hyperopt import hp, fmin, tpe
from vastai.api import VastClient
from toolbench.tools import print_module_versions
print_module_versions(globals())

Using TensorFlow backend.


             IPython  {'7.9.0'}
          tensorflow  {'1.15.0'}
             pymongo  {'3.9.0'}
 avian_vocalizations  {'0.0.1'}
            hyperopt  {'0.2.2'}
           toolbench  {'0.0.1'}


In [3]:
vast = VastClient().authenticate()

Initializing vast.ai client with api_key from /home/sam/.vast_api_key.


In [4]:
vast.get_running_instances()

[405882: running   running              $0.1031/hr ssh5.vast.ai:15882  259.7↑ 345.0↓  flops:14.056448T
 	1X 10.9GB GTX 1080 Ti, 5.0/40X 63.7GB Xeon® E5-2680 v2 
 	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter]

In [5]:
instance = vast.get_running_instances()[0]
# instance.start()
remote = instance.pb_remote
instance.ssh_connection_command

## Update code repo

In [8]:
with remote.cwd('/root/avian_vocalizations'):
    print(remote['git']('reset','--hard','HEAD^'))
    print(remote['git']('pull'))
    print(remote['git']('status'))

HEAD is now at 9cbb633 tensorflow 1.15.0. Added hyperopt trial reporting.

Updating 9cbb633..7adcfe0
Fast-forward
 HyperoptTrials.ipynb             |  32 -----
 Run Hyperopt Job.ipynb           | 271 ++++++++-------------------------------
 avian_vocalizations/evaluator.py |   5 +-
 3 files changed, 58 insertions(+), 250 deletions(-)

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	INSTALL_LOG
	logfile.txt
	start_hyperopt_worker.sh

nothing added to commit but untracked files present (use "git add" to track)



## Run some trials

In [9]:
tunnel = instance.get_tunnel(27018, 27017)

In [10]:
# trials.delete_all()

In [12]:
trials = MongoTrials('mongo://localhost:27018/avian_vocalizations/jobs', 
                     exp_key='alpha0')
hp_space = ParamSpace(n_frames=128, 
                      dropout_rate=hp.uniform('dropout',0,.5),
                      batch_size=hp.choice('batch_size',[64]),
                     )

evaluator = EvaluatorFactory(n_splits=2, n_epochs=100, data_dir='data')

best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=10)
best

over-writing old domain trials attachment


100%|██████████| 10/10 [3:40:44<00:00, 1324.41s/trial, best loss: 3.326384663581848] 


{'batch_size': 0, 'dropout': 0.03610653836571337}

In [ ]:
trials.trial_attachments(trials.best_trial).items()

In [ ]:
# trials.delete_all()

## Cleanup

In [ ]:
tunnel.close()

In [ ]:
# instance.stop()